In [26]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.nn import Linear, Sigmoid, ReLU
from torch.autograd import Variable

In [36]:
X_arr=[]
Y_arr=[]
#это пример с рандомными значениями
# Предположим нам надо предсказывать плотность загрузки по средним размерам коробки в контейнере
folder_path = "C:/Users/AliSa/Prakticum/ALGORITM"
directory_contents = os.listdir(folder_path)
for filename in directory_contents:
    df = pd.pandas.read_json("C://Users//AliSa//Prakticum//ALGORITM//"+filename)
    data = pd.DataFrame(df['data'])['data']
    cargoes = pd.DataFrame(list(data[0]))
    sizes = pd.DataFrame(list(cargoes['size']))
    sizes = sizes.iloc[:].values
    avg_width=0
    avg_height=0
    avg_length=0
    i=0
    for size in sizes:
        avg_width += size[0]
        avg_height += size[1]
        avg_length += size[2]
        i+=1
    avg_width /= i
    avg_height /= i
    avg_length /= i
    avg_all = (avg_width+avg_height+avg_length)/i
    avg_all = 1/avg_all
    X_arr.append([avg_width,avg_height,avg_length])
    data_result = pd.DataFrame(df['data_result'])['data_result']
    cargo_space = data_result['cargo_space']
    calculation_info = cargo_space['calculation_info']['density_percent']
    Y_arr.append(calculation_info)
X_arr= np.vstack(X_arr)
X_arr
print(Y_arr)
print(X_arr)

[81.76416, 82.5, 94.44444, 74.02778, 100.0, 92.20825, 55.72132, 82.7095, 58.69138, 55.72132, 100.0, 95.3125, 99.43182, 99.43182, 80.63824, 63.38312, 62.54826, 47.6821, 47.6821, 49.93126, 47.80008, 49.37232, 55.5809, 89.15496, 76.15648, 76.15648, 99.43182, 87.27273, 90.51218, 90.51218, 100.0, 82.084, 76.33322, 79.12946, 86.95652, 68.84688, 45.11657, 94.64265, 94.64265, 79.10819, 63.64827, 98.24176, 48.6333, 48.51001, 48.51001, 48.51001, 76.12721, 88.59972, 67.50455, 79.12094, 79.12094, 79.64019, 55.07126, 83.33333, 62.0, 100.0, 80.2589, 79.86997, 72.82762, 92.54325, 92.54325, 92.54325, 91.18541, 94.56265, 95.88089, 46.64721, 58.3393, 57.48137, 96.64382, 81.76416, 78.19096, 69.94074, 73.19202, 72.06703, 72.03363, 85.92912, 76.33322, 72.06703, 55.27211, 55.27211, 55.27211, 57.39796, 42.80376, 43.66653, 69.71802, 74.09029, 71.67931, 71.67931, 69.51437, 58.35529, 84.5985, 79.69262, 95.92491, 73.18317, 98.61134, 90.51218, 100.0, 100.0, 82.78877, 69.51437, 78.05612, 80.2589, 78.05612, 85.9009

In [37]:
# В библиотеке torch для обучения данные должны быть в формате тензоров
# соответственно разделяем
# затем приводим к формату
X = torch.autograd.Variable(torch.FloatTensor(X_arr))
Y = torch.autograd.Variable(torch.FloatTensor(Y_arr))
# количество признаков потом указывает на вход в нейроне
num_features = X.shape[1]

In [38]:
# Создаем сеть то есть Sequential - последовательность
# сюда внутрь можно добавлять разные слои
# документация - https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html
model = torch.nn.Sequential(
    Linear(num_features, 1),
    ReLU()
)

# Пока здесь один слой линейный - Linear
# ReLU() - функция активация, после слоя
# функция активация тоже могут быть разные - можно погулить

In [31]:
# задаем какую-нибудь функцию потерь для обучения
# здесь стандартная как для регрессии - квадратичная ошибка
loss_fn = torch.nn.MSELoss(size_average=False)

# шаг градиентного спуска (точнее -- метода оптимизации)
learning_rate = 0.00001  
# сам метод оптимизации нейросети (обычно лучше всего по-умолчанию работает Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

# итерируемся num_epochs раз, здесь 100000
for t in range(50000):
    # foward_pass() -- применение нейросети
    y_pred = model(X)

    # выведем loss - смотрим - ошибка на каждом этапе уменьшается
    loss = loss_fn(y_pred, Y)
    print('{} {}'.format(t, loss.data))

    # обнуляем градиенты перед backard_pass'ом (обязательно!)
    optimizer.zero_grad()

    # backward_pass() -- вычисляем градиенты loss'а по параметрам (весам) нейросети
    # этой командой мы только вычисляем градиенты, но ещё не обновляем веса
    loss.backward()

    # а тут уже обновляем веса
    optimizer.step()

C:\Users\AliSa\anaconda3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
C:\Users\AliSa\anaconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([628])) that is different to the input size (torch.Size([628, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0 2909483520.0
1 2909296896.0
2 2909110272.0
3 2908923904.0
4 2908737024.0
5 2908550656.0
6 2908364288.0
7 2908178432.0
8 2907992832.0
9 2907807232.0
10 2907621376.0
11 2907435520.0
12 2907249920.0
13 2907064832.0
14 2906879232.0
15 2906694144.0
16 2906508800.0
17 2906323456.0
18 2906138368.0
19 2905953792.0
20 2905768960.0
21 2905584640.0
22 2905399808.0
23 2905215232.0
24 2905030912.0
25 2904846592.0
26 2904662528.0
27 2904478720.0
28 2904294400.0
29 2904110592.0
30 2903923456.0
31 2903731968.0
32 2903540224.0
33 2903348480.0
34 2903155712.0
35 2902963200.0
36 2902770176.0
37 2902577664.0
38 2902384640.0
39 2902191616.0
40 2901998592.0
41 2901805824.0
42 2901612800.0
43 2901419776.0
44 2901227008.0
45 2901034240.0
46 2900841472.0
47 2900648960.0
48 2900455936.0
49 2900263680.0
50 2900071424.0
51 2899878912.0
52 2899686656.0
53 2899494400.0
54 2899302400.0
55 2899110400.0
56 2898918912.0
57 2898726912.0
58 2898535424.0
59 2898343680.0
60 2898152704.0
61 2897961216.0
62 2897770496.0
63

523 2820859392.0
524 2820712448.0
525 2820565760.0
526 2820417792.0
527 2820268288.0
528 2820118528.0
529 2819968768.0
530 2819819008.0
531 2819668736.0
532 2819518464.0
533 2819368448.0
534 2819218432.0
535 2819068160.0
536 2818918144.0
537 2818767872.0
538 2818617856.0
539 2818467328.0
540 2818317312.0
541 2818167296.0
542 2818016768.0
543 2817867008.0
544 2817716992.0
545 2817567232.0
546 2817417216.0
547 2817267456.0
548 2817117440.0
549 2816968192.0
550 2816818688.0
551 2816668672.0
552 2816519424.0
553 2816370176.0
554 2816220672.0
555 2816071680.0
556 2815922432.0
557 2815773184.0
558 2815624192.0
559 2815475712.0
560 2815326976.0
561 2815177984.0
562 2815029760.0
563 2814881024.0
564 2814732800.0
565 2814584576.0
566 2814436352.0
567 2814288384.0
568 2814140160.0
569 2813992448.0
570 2813844224.0
571 2813696512.0
572 2813549056.0
573 2813401088.0
574 2813253632.0
575 2813106176.0
576 2812958720.0
577 2812811776.0
578 2812664576.0
579 2812517632.0
580 2812370944.0
581 2812224000

1028 2754318848.0
1029 2754202112.0
1030 2754085376.0
1031 2753968896.0
1032 2753852416.0
1033 2753736192.0
1034 2753619712.0
1035 2753503232.0
1036 2753387520.0
1037 2753271296.0
1038 2753155584.0
1039 2753039360.0
1040 2752923648.0
1041 2752807936.0
1042 2752692480.0
1043 2752577024.0
1044 2752461312.0
1045 2752346112.0
1046 2752230656.0
1047 2752115456.0
1048 2752000512.0
1049 2751885312.0
1050 2751769856.0
1051 2751654912.0
1052 2751539712.0
1053 2751425280.0
1054 2751310336.0
1055 2751195392.0
1056 2751080960.0
1057 2750966528.0
1058 2750851840.0
1059 2750737664.0
1060 2750623488.0
1061 2750509568.0
1062 2750394880.0
1063 2750281216.0
1064 2750167296.0
1065 2750053376.0
1066 2749939456.0
1067 2749825536.0
1068 2749712128.0
1069 2749598208.0
1070 2749484544.0
1071 2749370880.0
1072 2749257728.0
1073 2749144320.0
1074 2749030912.0
1075 2748917760.0
1076 2748804608.0
1077 2748691200.0
1078 2748578304.0
1079 2748465664.0
1080 2748352768.0
1081 2748240384.0
1082 2748127744.0
1083 27480

1485 2699292928.0
1486 2699181824.0
1487 2699070464.0
1488 2698959616.0
1489 2698848768.0
1490 2698737664.0
1491 2698626560.0
1492 2698515968.0
1493 2698405120.0
1494 2698294272.0
1495 2698183680.0
1496 2698073344.0
1497 2697963008.0
1498 2697852160.0
1499 2697741824.0
1500 2697631488.0
1501 2697521152.0
1502 2697410816.0
1503 2697300736.0
1504 2697190400.0
1505 2697080320.0
1506 2696970240.0
1507 2696860672.0
1508 2696750592.0
1509 2696641024.0
1510 2696531456.0
1511 2696421632.0
1512 2696312064.0
1513 2696202496.0
1514 2696093184.0
1515 2695983616.0
1516 2695874304.0
1517 2695764992.0
1518 2695655680.0
1519 2695546624.0
1520 2695437568.0
1521 2695328512.0
1522 2695219712.0
1523 2695110656.0
1524 2695001856.0
1525 2694893056.0
1526 2694784256.0
1527 2694675456.0
1528 2694566912.0
1529 2694458368.0
1530 2694349824.0
1531 2694241024.0
1532 2694132736.0
1533 2694024704.0
1534 2693916416.0
1535 2693808128.0
1536 2693700096.0
1537 2693592064.0
1538 2693484032.0
1539 2693376000.0
1540 26932

...

48007 1737769216.0
48008 1737687168.0
48009 1737604480.0
48010 1737521920.0
48011 1737439744.0
48012 1737358464.0
48013 1737277952.0
48014 1737198592.0
48015 1737120384.0
48016 1737040896.0
48017 1736962176.0
48018 1736884864.0
48019 1736808448.0
48020 1736733440.0
48021 1736659072.0
48022 1736585472.0
48023 1736512768.0
48024 1736439808.0
48025 1736363008.0
48026 1736286208.0
48027 1736209920.0
48028 1736133632.0
48029 1736057600.0
48030 1735981568.0
48031 1735905408.0
48032 1735829376.0
48033 1735753216.0
48034 1735677056.0
48035 1735600896.0
48036 1735524608.0
48037 1735448192.0
48038 1735371648.0
48039 1735295104.0
48040 1735218304.0
48041 1735141632.0
48042 1735064704.0
48043 1734988032.0
48044 1734911104.0
48045 1734834432.0
48046 1734757376.0
48047 1734680576.0
48048 1734603776.0
48049 1734527232.0
48050 1734450560.0
48051 1734373888.0
48052 1734297472.0
48053 1734220800.0
48054 1734144384.0
48055 1734068224.0
48056 1733992192.0
48057 1733915904.0
48058 1733840000.0
48059 173376

48461 1704604160.0
48462 1704536320.0
48463 1704468480.0
48464 1704400640.0
48465 1704332544.0
48466 1704264704.0
48467 1704196864.0
48468 1704129024.0
48469 1704061184.0
48470 1703993344.0
48471 1703925504.0
48472 1703857792.0
48473 1703790080.0
48474 1703722368.0
48475 1703654528.0
48476 1703586816.0
48477 1703518976.0
48478 1703451648.0
48479 1703383936.0
48480 1703316224.0
48481 1703248128.0
48482 1703179264.0
48483 1703110144.0
48484 1703040896.0
48485 1702971392.0
48486 1702901888.0
48487 1702832256.0
48488 1702762624.0
48489 1702692864.0
48490 1702623232.0
48491 1702553728.0
48492 1702484224.0
48493 1702414592.0
48494 1702345088.0
48495 1702275840.0
48496 1702206592.0
48497 1702137216.0
48498 1702067968.0
48499 1701998720.0
48500 1701929472.0
48501 1701860352.0
48502 1701791360.0
48503 1701721472.0
48504 1701651328.0
48505 1701580800.0
48506 1701510144.0
48507 1701439488.0
48508 1701368704.0
48509 1701298048.0
48510 1701227392.0
48511 1701156864.0
48512 1701086208.0
48513 170101

48924 1671213056.0
48925 1671130112.0
48926 1671047296.0
48927 1670964480.0
48928 1670881536.0
48929 1670799104.0
48930 1670716928.0
48931 1670634752.0
48932 1670552832.0
48933 1670471296.0
48934 1670389504.0
48935 1670308352.0
48936 1670227072.0
48937 1670146048.0
48938 1670065152.0
48939 1669984256.0
48940 1669903616.0
48941 1669822976.0
48942 1669742464.0
48943 1669661696.0
48944 1669581312.0
48945 1669500800.0
48946 1669420160.0
48947 1669339776.0
48948 1669259264.0
48949 1669178880.0
48950 1669098496.0
48951 1669018368.0
48952 1668937856.0
48953 1668857856.0
48954 1668777728.0
48955 1668697728.0
48956 1668617728.0
48957 1668537856.0
48958 1668457984.0
48959 1668378240.0
48960 1668298496.0
48961 1668218624.0
48962 1668139264.0
48963 1668059904.0
48964 1667980160.0
48965 1667900800.0
48966 1667821568.0
48967 1667742208.0
48968 1667662848.0
48969 1667583872.0
48970 1667504640.0
48971 1667425664.0
48972 1667346688.0
48973 1667267584.0
48974 1667188608.0
48975 1667109760.0
48976 166703

49369 1635779584.0
49370 1635701376.0
49371 1635623168.0
49372 1635544832.0
49373 1635466752.0
49374 1635388672.0
49375 1635310464.0
49376 1635232512.0
49377 1635154432.0
49378 1635076224.0
49379 1634998272.0
49380 1634919936.0
49381 1634842112.0
49382 1634764160.0
49383 1634686208.0
49384 1634608384.0
49385 1634530560.0
49386 1634452608.0
49387 1634374656.0
49388 1634296960.0
49389 1634219136.0
49390 1634141312.0
49391 1634063616.0
49392 1633985792.0
49393 1633908224.0
49394 1633830400.0
49395 1633751808.0
49396 1633672832.0
49397 1633594112.0
49398 1633515008.0
49399 1633436160.0
49400 1633357184.0
49401 1633278080.0
49402 1633199232.0
49403 1633120256.0
49404 1633041408.0
49405 1632962560.0
49406 1632884096.0
49407 1632805632.0
49408 1632727168.0
49409 1632648576.0
49410 1632570112.0
49411 1632491776.0
49412 1632413440.0
49413 1632335104.0
49414 1632256896.0
49415 1632178432.0
49416 1632099840.0
49417 1632021760.0
49418 1631943296.0
49419 1631864832.0
49420 1631786624.0
49421 163170

49830 1600035456.0
49831 1599953024.0
49832 1599870336.0
49833 1599787392.0
49834 1599704320.0
49835 1599620608.0
49836 1599536896.0
49837 1599452928.0
49838 1599368832.0
49839 1599284608.0
49840 1599200384.0
49841 1599116416.0
49842 1599032320.0
49843 1598948352.0
49844 1598864640.0
49845 1598780800.0
49846 1598697216.0
49847 1598613888.0
49848 1598530304.0
49849 1598446720.0
49850 1598362880.0
49851 1598278912.0
49852 1598195456.0
49853 1598111872.0
49854 1598028544.0
49855 1597945344.0
49856 1597862656.0
49857 1597779968.0
49858 1597697536.0
49859 1597615360.0
49860 1597533312.0
49861 1597451264.0
49862 1597369600.0
49863 1597287936.0
49864 1597206272.0
49865 1597124864.0
49866 1597043328.0
49867 1596961920.0
49868 1596880640.0
49869 1596799104.0
49870 1596717824.0
49871 1596636416.0
49872 1596555008.0
49873 1596473728.0
49874 1596392448.0
49875 1596311296.0
49876 1596230016.0
49877 1596148736.0
49878 1596067584.0
49879 1595986560.0
49880 1595905408.0
49881 1595824384.0
49882 159574

In [33]:
# Теперь можно посмотреть как можель прогнозирует
# Допустим есть вот такие параметры коробок - 1520, 1200, 1414
x = [[137.02475248, 180.57920792, 228.21287129]]
# также приводим к тензору
x = torch.autograd.Variable(torch.FloatTensor(x))
# смотрим какую плотность модель предсказала
c = model(x)
c
print(c)


tensor([[4.5188]], grad_fn=<ReluBackward0>)
